### Import library

In [16]:
import numpy as np
import pandas as pd
import lightgbm as lgb

### Load data

In [17]:
df = pd.read_csv('train.csv', nrows=22000000)
df = df.drop('ID',axis=1)
df['F04'] = df['F04'].fillna(0)
df['F11'] = df['F11'].fillna(0)
df['F18'] = df['F18'].fillna(0)
df['F19'] = df['F19'].fillna(0)
df['F24'] = df['F24'].fillna(0)
df['F27'] = df['F27'].fillna(0)
df['F29'] = df['F29'].fillna(0)
df['F32'] = df['F32'].fillna(0)
df['F33'] = df['F33'].fillna(0)
df['F36'] = df['F36'].fillna(0)
df['F38'] = df['F38'].fillna(0)
df = df.fillna('NAN')
float_columns = df.select_dtypes(include=['float64']).columns
df[float_columns] = df[float_columns].astype('int64')
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype('category')
df.info()

MemoryError: Unable to allocate 128. KiB for an array with shape (16384,) and data type int64

In [ ]:
model = lgb.LGBMClassifier(
    objective='binary',
    learning_rate=0.1,
    num_leaves=80,         # 적당히 작은 값
    max_depth=30,          # 적당히 작은 값
    min_data_in_leaf=100,
    feature_fraction=1.0,
    bagging_fraction=1.0,
    bagging_freq=8,
    boosting_type='gbdt',
    max_bin=500,           # 기본값 255에서 줄여볼 수 있음 (예: 127)
    subsample=1.0,         # 각 트리 학습에 사용하는 데이터 비율
    colsample_bytree=1.0   # 각 트리 학습에 사용하는 특성 비율
)
model.fit(df.drop('Click',axis=1),df['Click'],eval_metric='AUC')

### Load test data

In [ ]:
def load_data():
    df = pd.read_csv('test.csv')
    df = df.drop('ID',axis=1)
    df['F04'] = df['F04'].fillna(0)
    df['F11'] = df['F11'].fillna(0)
    df['F18'] = df['F18'].fillna(0)
    df['F19'] = df['F19'].fillna(0)
    df['F24'] = df['F24'].fillna(0)
    df['F27'] = df['F27'].fillna(0)
    df['F29'] = df['F29'].fillna(0)
    df['F32'] = df['F32'].fillna(0)
    df['F33'] = df['F33'].fillna(0)
    df['F36'] = df['F36'].fillna(0)
    df['F38'] = df['F38'].fillna(0)
    df = df.fillna('NAN')
    float_columns = df.select_dtypes(include=['float64']).columns
    df[float_columns] = df[float_columns].astype('int64')
    object_columns = df.select_dtypes(include=['object']).columns
    df[object_columns] = df[object_columns].astype('category')
    return df


test_df = load_data()

### Infer

In [ ]:
pred = model.predict_proba(test_df)

### Submit

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission

sample_submission['Click'] = pred[:,1]
sample_submission

sample_submission.to_csv('lgbm.csv', index=False)